# Authors of Interest

Created in 2015-03-25 by Alberto Ueda

In [1]:
%pylab inline
%matplotlib inline

import pandas as pd
import numpy as np
import mmap
import time
import re

from unidecode import unidecode
UTF8 = 'utf-8'        

class Dict(dict):
    def __missing__(self, key):
            return False

#reload(sys)  # Solve the acentos problem, but create the terminal output problem
#sys.setdefaultencoding(UTF8)

mypath = '/mnt/hd0/alberto/Dropbox/ufmg-not-code/datasets/dblp/dblp-cuts/'
# mypath = '/home/alberto/Dropbox/ufmg-not-code/datasets/dblp/dblp-cuts/'

Populating the interactive namespace from numpy and matplotlib


In [8]:
a_p = pd.read_csv(mypath + "../authorpaper.csv", names=['author','paper'])#, nrows=17220)
a_p.head()

,author,paper
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


## Counting Publications

In [44]:
# Exec Time: 1 min
p_counts = a_p.groupby('author').count()
p_counts.columns = ['n_papers']
p_counts.to_csv(mypath + 'author_num_papers.csv')

In [32]:
p_counts.ix[0]['paper'] # Expected: 1095

1095

In [38]:
len(p_counts[p_counts['paper'] < 5])

1182692

## Counting Coauthorships


In [2]:
a_p = pd.read_csv(mypath + "../authorpaper.csv", names=['author','paper'])#, nrows=17220)
a_p.head()

,author,paper
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [3]:
# this block takes a long time to run (5min)
groups = a_p.groupby('paper')['author'].apply(list) 
groups.head()

paper
0                        [0, 189099]
1           [0, 2029, 44493, 246705]
2                 [0, 22572, 111477]
3                            [0, 21]
4    [0, 4590, 10861, 20004, 258270]
Name: author, dtype: object

In [5]:
# Saving output...
groups.to_csv(mypath + 'paper-authors.csv')

In [10]:
# Really counting
count = Dict()

def connect_coauthors(coauthors):
    for author1 in coauthors: 
        for author2 in coauthors:
            if author1 < author2:                                 
                key = str(author1) + ',' + str(author2);
                if count[key] == False: 
                    count[key] = 1
                else:
                    count[key] += 1                                                        
                # if count[key] > 1: print (key, ' --> ', count[key])               
                 
k = 0
for paper_coauthors in groups.values:
    #if k > 29: break  #max:4551362 # this takes a long time to run
    k += 1
    connect_coauthors(paper_coauthors)            
        
# with open(mypath + 'coauthorships.csv', 'w') as f:    

In [13]:
# Writing in file
coauthorships = pd.DataFrame.from_dict(list(count.items()))
coauthorships.columns = ['author1,author2', 'num_papers']
coauthorships.to_csv(mypath + 'coauthorships.csv', index=False)

# Remove double quotes with: $ sed -i 's/\"//g' coauthorships.csv (30s)